# Cifrado Flujo

## Ejercicio 1

Escribe una función que determine si una secuencia de bits cumple los postulados de Golomb.

In [1]:
from math import ceil, floor
bits = [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]

def first_rule(seq):
    # Contamos el número de unos
    n_ones, n_bits = sum(seq), len(seq)
    # Si la longitud de la secuencia es 0
    if n_bits % 2 == 0:
        # a la fuerza tienen que ser iguales
        # num_ceros y num_unos
        return n_bits // 2 == n_ones
    else:
        n2_bits = n_bits / 2
        return ceil(n2_bits) == n_ones or floor(n2_bits) == n_ones

    
first_rule(bits)

True

En esta primera regla, podemos hacer una rápida comprobación si sumamos la secuencia de bits y obtenemos su longitud. Si la longitud es par, tenemos que el número de unos tiene que ser $\frac{|| bits ||}{2}$, y en caso de qeu sea impar, el número de unos tiene que ser igual a $\left\lfloor\frac{|| bits ||}{2}\right\rfloor$ o $\left\lceil\frac{|| bits ||}{2}\right\rceil$. En caso contrario, devolveremos un `False`.

In [2]:
from itertools import groupby
from collections import Counter

# Esta función calcula el número de rachas
# de longitud k en la secuencia seq      
def second_rule(seq):
    while seq[0] == seq[-1]:
        seq.append(seq.pop(0))
        
    # obtenemos todas las rachas posibles que existen en la secuencia
    runs = [list(g) for k, g in groupby(seq)]
    # y contamos el número de rachas que hay para cada longitud
    count = Counter(map(lambda x: len(x), runs))
    # comprobamos si se cumple que #runs(k+1) == runs(k)
    for i in range(1, len(count)):
        if count[i] != count[i+1]:
            if not count[i] >= 2*count[i+1]: # en el caso de que el siguiente elemento
                return False                 # no sea 1/2 veces más pequeño
    else:
        # si todo va bien, devolvemos True
        return True

second_rule(bits)

True

Para la segunda regla, comprobamos el número de rachas de longitud $k$ debe ser el doble que el número de rachas de longitud $k+1$ en la secuencia de bits, aunque esta condición se relaja un poco. En caso de que en algún momento esto no se cumpla, se devuelve `False`.

In [3]:
from numpy import bitwise_xor, nonzero

def rotate(seq, length):
    seq = [seq[-1]] + seq[:length-1]

def third_rule(seq):
    length, rotated_seq = len(seq), seq
    rotate(rotated_seq, length)
    # Calculamos la distancia hamming como un xor entre
    # ambas cadenas, y contamos los bits no nulos
    norm = len(bitwise_xor(seq, rotated_seq).nonzero()[0])
    for i in range(1, length):
        rotate(rotated_seq, length)
        if norm != len(bitwise_xor(seq, rotated_seq).nonzero()[0]):
            return False
    else:
        return True

third_rule(bits)

True

En esta regla, comprobamos si, rotando la cadena de bits, se mantiene constante la distancia Hamming. Esta distancia Hamming se calcula como la suma de los bits no nulos resultantes de hacer un $xor$ entre la cadena original y la cadena rotada.

In [4]:
def Golomb(seq):
    if all(seq):
        return False
    else:
        rules = [first_rule, second_rule, third_rule]
        return all(map(lambda x: rules[x](seq), range(3)))
    
Golomb(bits)

True

Esta función comprueba aplica los postulados de Golomb para determinar si una secuencia de bits es lo suficientemente aleatoria. Primeramente, comprueba que todos los elementos sean distintos y si lo son, pasa a aplicar cada uno de los postulados para comprobar si es correcto o no.

## Ejercicio 2

Implementa registros lineales de desplazamiento con retroalimentación. La entrada son los coeficientes del polinomio de conexión, la semilla, y la longitud de la secuencia de salida.

Ilustra con ejemplos la dependencia del periodo de la semilla en el caso de polinomios reducibles, la independencia en el caso de polinomios irreducibles, y la maximalidad del periodo en el caso de polinomios primitivos.

Comprueba que los ejemplos con polinomios primitivos satisfacen los postulados de Golomb.

In [5]:
from numpy import bitwise_and
# Linear Feedback Shift Register
def LFSR(conex_poly, seed, length):
    register = list(seed)
    out = [0]*length
    # obtenemos el grado del polinomio de conexión
    for i in range(length):
        register.append((len(bitwise_and(conex_poly, register).nonzero()[0])) % 2)
        out[i] = register.pop(0)
    
    return out

Para demostrar la dependencia del periodo de la semilla según el tipo de polinomio, vamos a verlos uno por uno. 
* __Polinomios reducibles__: para demostrar la dependencia en el periodo, definimos el siguiente polinomio $c(D) = D^4 + D^2 + 1$, y como semilla, mandaremos las siguientes cadenas: $c_1 = [1,0,0,1]$ y $c_2 = [1,1,0,1]$.

In [6]:
n = 24
print("Cadena 1:", LFSR([1,0,1,0], [1,0,0,1], n))
print("Cadena 2:", LFSR([1,0,1,0], [1,1,0,1], n))

Cadena 1: [1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
Cadena 2: [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]


Como podemos ver, en el caso de la cadena $c_1$, el periodo tiene una longitud de 6 elementos: [1,0,0,1,1,1]. A partir de aquí, esta lista se va repitiendo constantemente, por lo que tenemos un periodo muy pequeño. 

Esto es aún peor en el caso de la cadena $c_2$, donde tenemos un período de longitud tres, siendo [1,1,0]. 

Con esto podemos ver cómo el periodo es altamente dependiente del poliniomio que establezcamos. Además, nunca podremos alcanzar el periodo máximo con un polinomio reducible.

* __Polinomios irreducibles__: en este caso, vamos a introducir el siguiente polinomio irreducible para grado 3: $c(D) = D^3 + D + 1$, que es irreducible. Como semilla, introduciremos la cadena $c = [1,1,0,0]$.

In [7]:
print("Cadena 3:", LFSR([0,1,0,1], [1,1,0,0], n))

Cadena 3: [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0]


Como podemos ver, en este caso, el período es de longitud 6, siendo este periodo [0, 1, 1, 1, 0, 1]. Como se puede ver, el período aún sigue siendo pequeño y no llega al máximo posible. Esto se debe a que el polinimio, es degenerado para el grado 4. 

* __Poliniomios primitivos__: en este caso, probaremos con el poliniomio primitivo $c(D) = D^4 + D + 1$ y la semilla, introduciremos la cadena $c = [0,0,0,1]$.

In [8]:
LFSR([1,0,0,1], [0,0,0,1], n)

[0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]

En el caso de los polinomios primitivos, alcanzamos el período máximo, que es $2^4 = 16$, siendo este período la cadena [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1].En este caso vemos que cuanto mayor sea el grado del polinomio y la longitud de la cadena, mayor será la longitud del periodo.

Para comprobar si las cadenas obtenidas con el LFSR y un polinomio irreducible cumplen los postulados de Golomb, podemos hacer lo siguiente:

In [9]:
polyms = [[1,0,1], [1,0,0,1], [1,0,0,1,0], [1,0,0,0,0,1],
          [1,0,0,0,0,0,1], [1,0,0,0,0,1,0,0,0]]

seeds = [[0,1,1], [0,0,0,1], [0,1,1,1,0], [0,1,1,1,0,0], 
        [0,0,0,1,1,0,1], [0,1,0,1,0,1,0,0,1]]

for polym, seed in zip(polyms, seeds):
    result = LFSR(polym, seed, 2**len(polym)-1)
    if not Golomb(result):
        print("La lista que forma la semilla", seed, 
              " con el polinomio", polym, "\n no cumple los postulados.")

Como podemos observar, las cadenas obtenidas con polinomios irreducibles, cumplen los postulados de Golomb, ya que al ejecutar el código anterior, la salida ha sido vacía.

## Ejercicio 3

Un polinomio en varias variables con coeficientes en $\mathbb{Z}_2$ se puede expresar como suma de monomios, simplemente usando la propiedad distributiva. Cualquier monomio $x_1^{e_1}\cdots x_n^{e_n}, e_i \in \mathbb{N}$, es, como función, equivalente a un monomio de la forma $x_{i_1}\cdots x_{i_r} (x^2 = x$ $\forall x \in \mathbb{Z}_2$, los $i_j$ son precisamente los índices tales que $e_{i_j} \neq 0)$. Por ejemplo, $1 + x^2(y + x) = 1 + x^3 + x^2y$, esta expresión es equivalente a $1 + x +xy$, por lo que la repreentamos mediante [[0,0], [1,0], [1,1]], que se corresponde con la lista de exponentes en las dos variables: $x^0y^0 + x^1y^0 + x^1y^1$. Así un polinomio en $\mathbb{Z}_2$ se puede representar por una lista de monomios. Y cada monomio como una lista de 0 y 1, que corresponden con los exponentes de cada una de las variables que intervienen en el polinomio.

Escribe una función que toma como argumentos una función polinómica $f$, una semilla $s$ y un entero positivo $k$, y devuelve una secuencia de longitud $k$ generada al aplicar a $s$ el registro no lineal de desplazamiento con retroalimentación asociado a $f$.

Encuentra el periodo de la $NLFSR((x \wedge y) \vee \bar{z} \oplus t)$ con semilla 1011.

In [10]:
f = lambda seed: ((seed[0] & seed[1]) | (not seed[2])) ^ seed[3]

def NLFSR(function, seed, length):
    out = [0]*length
    # obtenemos el grado del polinomio de conexión
    for i in range(length):
        seed.append(function(seed)) # añadimos el resultado al final
        out[i] = seed.pop(0) # y devolvemos el primer bit
    
    return out


NLFSR(f, [1,0,1,1], 20)

[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]

En este caso, podemos ver que la salida de la función anterior con la semilla [1,0,1,1] tiene un periodo de longitud 3 en el que se repite la subcadena [1,0,0]. Además de esto, la función original $(x \wedge y) \vee \bar{z} \oplus t$, podemos calcular la forma canónica para obtener un polinimo con los coeficientes:

## Ejercicio 4

Implementa el generador de Geffe. Encuentra ejemplos donde el periodo de la salida es $p_1p_2p_3$, con $p_1,p_2$ y $p_3$ los periodos de los tres LFSRs usados en el generador de Geffe.

Usa este ejercicio para construir un cifrado en flujo. Con entrada un mensaje $m$, construye una llave $k$ con la misma longitud que $m$ y devuelve $m \oplus k$ (donde $\oplus$ significa suma componente a componente en $\mathbb{Z}_2$).

El descifrado se hace de la misma forma: $c \oplus k$ (nótese que $c\oplus k = (m\oplus k) \oplus k = m (k \oplus k) = m$ ya que $x \oplus x = 0$ en $\mathbb{Z}_2$).

In [11]:
from numpy import bitwise_not

# Conver a str to a bin list like [0,1,0,0,1...]
def str_to_binlist(message):
    # get a binary list
    binary = list(bin(int.from_bytes(message.encode('utf-8'), byteorder='big'))) 
    # return the stream of 0 and 1
    binary.remove('b')
    return list(map(lambda x: int(x), binary))


def bin_to_str(bin_message):
    # pass elements from int to str 
    message = list(map(lambda x: str(x), bin_message))
    # create a full str
    complete_msg = ''.join(message)
    # converte to str using a integer in Z_2
    int_msg = int(complete_msg,2)
    return int_msg.to_bytes((int_msg.bit_length() + 7) // 8, 
                            byteorder='big').decode('utf-8', 'ignore')
    
    
def Geffe(arg1, arg2, arg3, length):
    # Calculate the LFSRs with their polynoms
    # and seeds with a fixed length
    LFSR1 = LFSR(arg1[0], arg1[1], length)
    LFSR2 = LFSR(arg2[0], arg2[1], length)
    LFSR3 = LFSR(arg3[0], arg3[1], length)
    # Return the complete sequence
    return Geffe_func(LFSR1, LFSR2, LFSR3)

    
def Geffe_func(LFSR1, LFSR2, LFSR3):
    # LFSR1 and LFSR2
    result1 = bitwise_and(LFSR1, LFSR2)
    # not LFSR2 and LFSR3
    result2 = bitwise_and(bitwise_not(LFSR2), LFSR3)
    # result1 and result2
    return bitwise_xor(result1, result2)


# encrypt message using Geffe
def Geffe_encrypt(key, message):
    # get the binary list
    bin_message = str_to_binlist(message)
    # compute the Geffe sequence
    element = Geffe(key[0], key[1], key[2], len(bin_message))
    # encrypt the message
    cypher = bitwise_xor(element, bin_message)
    # return the cypher text
    return cypher


# decrypt message using Geffe 
def Geffe_decrypt(key, cypher):
    # compute the Geffe sequence
    element = Geffe(key[0], key[1], key[2], len(cypher))
    # Decrypt the message
    return bin_to_str(bitwise_xor(element,cypher))


# Key to cypher text
key = [([1,1,0,0,1,0], [1,1,1,1,0,1]), ([1,0,1,0,1,1], [1,0,1,1,1,1]), ([1,1,0,1,0,0], [1,1,0,1,0,0])]

encrypt = Geffe_encrypt(key, 'El poder desgasta a quien no lo tiene')
decrypt = Geffe_decrypt(key, encrypt)
print("Cypher text:", bin_to_str(encrypt))
print("Decypher text:", decrypt)

Cypher text: oe!&	sӿ_X**s-}
Decypher text: El poder desgasta a quien no lo tiene


Para realizar el cifrado basado en Geffe, se toma el generador de este para calcular la llave simétrica a partir de tres polinomios y tres semillas distintas. La longitud de esta clave será igual a la longitud del mensaje en binario, siendo calculada esta cadena por *str_to_binlist*. 

Una vez calculada esta cadena y la clave, realizamos la operación $m \oplus k$, donde $m$ es el mensaje en plano y $k$ es la clave generada gracias al generador de Geffe. De esta operación, obtenemos el mensaje cifrado en binario $c$ que será lo que devuelve la función *Geffe_encrypt*.

Este mensaje cifrado en binario, será la cadena que recibe el destinatario del mensaje, que pasará a descifrar con la función _Geffe_decrypt_. El destinatario computa la llave con el generador de Geffe y descifra el mensaje haciendo la misma operación que para encriptar, pero con el mensaje cifrado $c \oplus k = m$. A diferencia de la función anterior, la función de descifrado nos devuelve el mensaje en formato _human readable_, gracias a la función *bin_to_str*.

## Ejercicio 5

Dada una sucesión de bits periódica, determina la complejidad lineal de dicha sucesión, y el polinomio de conexión que la genera.  Para esto, usa el algoritmo de Berlekamp-Massey.

Haz ejemplos con sumas y productos de secuencias para ver qué ocurre con la complejidad lineal.

In [12]:
def Berlekam_Massey(sequence):
    # initialize the parameters 
    n_bits = len(sequence)
    C, B, L, m = [0]*n_bits, [0]*n_bits, 0, -1
    C[0], B[0] = 1,1
    
    for N in range(n_bits):
        # Compute the next discrepancy
        di = (sequence[N] + sum([sequence[N - i]*C[ i] for i in range(1, L + 1)])) % 2
        if di == 1: # Update register
            T, D = C, list(B)
            # D = B * X^(N-m) (add zeros to left side)
            for i in range(N-m):
                D.insert(0,0); D.pop(-1) # remove the trash bits
            # C = C + B * X^(N-m)
            C = bitwise_xor(C, D)
            # Update the parameters
            if L <= N // 2:
                L, m, B = N +1 - L, N, T
                
    # return linear complexity and the polynom ready to use
    finalC = list(C[::-1])
    while finalC[0] == 0:
        finalC.pop(0)
    
    return L, finalC


sequence = [0,0,1,1,0,1,1,1,0]
linear_complexity, polyn = Berlekam_Massey(sequence)
print("Polinomio:", polyn, "\tComplejidad lineal:", linear_complexity)
print(polyn[:5])

Polinomio: [1, 0, 1, 0, 0, 1] 	Complejidad lineal: 5
[1, 0, 1, 0, 0]


Para resolver este ejercicio, he seguido el algoritmo que viene descrito en el libro "_Handbook of Applied Criptography_", donde en $n$ iteraciones calcula la complejidad lineal de la secuencia y el polinomio característico que genera esta secuencia.

En este caso, como durante el desarrollo del guión los polinomios se han tomado de la siguiente forma $C(x) = X^na_n + x^{n-1}a_{n-1} + \cdots + 1$ la función, una vez que ha obtenido el polinomio, lo invierte y elimina los ceros que se encuentra al principio para devolver el resultado. Para probar que está bien, podemos comprobar que el LFSR generado con este polinomio genera la secuencia anterior.

In [13]:
test_seq = LFSR(polyn[:5], sequence[:5], len(sequence))
print("Son iguales:", test_seq == sequence)

Son iguales: True
